In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.special import expit, logit
from scipy.stats import norm

# Dataset

In [94]:
df = pd.read_csv("bank-note/bank-note/train.csv", header=None)
d = df.to_numpy()
X = d[:,:-1]
Y = d[:,-1]

X.shape, Y.shape

((872, 4), (872,))

In [95]:
df = pd.read_csv("bank-note/bank-note/test.csv", header=None)
d = df.to_numpy()
Xtest = d[:,:-1]
Ytest = d[:,-1]

Xtest.shape, Ytest.shape

((500, 4), (500,))

# Part 1

In [96]:
def initialise_w(initialise):
    if(initialise == 'random'):
        w = np.random.randn(d,1)
        print("w is initialised from N[0,1]")
    elif(initialise == 'zeros'):
        w = np.zeros((d,1))
        print("w is initialised as a zero vector")
    else:
        print("Method unknown")
    return w

def compute_mu(X, w):
    mu = expit(np.dot(X,w))
    mu = mu.reshape(X.shape[0],1)
    return mu

def first_derivative(w):
    mu = compute_mu(X, w)
    epsilon = 1e-12

    grad = np.matmul(np.transpose(X), (mu-Y)) + w.reshape(d,1)
    grad = grad.squeeze()
    return(grad)

def second_deivative(w,X,y):
    mu = compute_mu(X, w)
    R = np.eye(n)
    for i in range(n):
        R[i,i] = mu[i,0] * (1-mu[i,0])
    return(np.dot(np.dot(np.transpose(X),R),X) + np.eye(d))

def test(w, X, y):
    n,d = X.shape
    mu = compute_mu(X, w)
    yhat = np.zeros((n,1)).astype(np.float64)
    yhat[mu>0.5]=1
    correct = np.sum(yhat==y)
    return(correct,n)

def train(initialise):

    np.random.seed(0)
    w = initialise_w(initialise)
    for j in range(100):

        grad1 = first_derivative(w.squeeze()).reshape(d,1)
        H = second_deivative(w, X, Y)
        delta_w = np.dot(np.linalg.inv(H),grad1)
        w = w - delta_w
        diff = np.linalg.norm(delta_w)

        correct,n = test(w, Xtest, Ytest)
        print("Iteration : {} \t Accuracy : {}%".format(j,correct/n*100))

        if(diff < 1e-5):
            print("tolerance reached at the iteration : ",j)
            break
    print("Training done...")
    print("Model weights : ", np.transpose(w))

In [97]:
n,d = X.shape
n1,d1 = Xtest.shape

Y = Y.reshape(n,1)
Ytest = Ytest.reshape(n1,1)

train('random')

w is initialised from N[0,1]
Iteration : 0 	 Accuracy : 76.0%
Iteration : 1 	 Accuracy : 34.8%
Iteration : 2 	 Accuracy : 81.2%
Iteration : 3 	 Accuracy : 55.800000000000004%
Iteration : 4 	 Accuracy : 47.199999999999996%
Iteration : 5 	 Accuracy : 63.800000000000004%
Iteration : 6 	 Accuracy : 49.0%
Iteration : 7 	 Accuracy : 63.6%
Iteration : 8 	 Accuracy : 49.0%
Iteration : 9 	 Accuracy : 63.6%
Iteration : 10 	 Accuracy : 49.0%
Iteration : 11 	 Accuracy : 63.6%
Iteration : 12 	 Accuracy : 49.0%
Iteration : 13 	 Accuracy : 63.6%
Iteration : 14 	 Accuracy : 49.0%
Iteration : 15 	 Accuracy : 63.6%
Iteration : 16 	 Accuracy : 49.0%
Iteration : 17 	 Accuracy : 63.6%
Iteration : 18 	 Accuracy : 49.0%
Iteration : 19 	 Accuracy : 63.6%
Iteration : 20 	 Accuracy : 49.0%
Iteration : 21 	 Accuracy : 63.6%
Iteration : 22 	 Accuracy : 49.0%
Iteration : 23 	 Accuracy : 63.6%
Iteration : 24 	 Accuracy : 49.0%
Iteration : 25 	 Accuracy : 63.6%
Iteration : 26 	 Accuracy : 49.0%
Iteration : 27 	 Accu

# Part 2

In [99]:
# LBFGS
def compute_mu(X, w):
    phi=np.dot(X,w)
    mu = norm.cdf(phi)
    mu = mu.reshape(X.shape[0],1)
    return mu

def first_derivative(w):
    mu = compute_mu(X, w)
    epsilon = 1e-12
    phi=np.dot(X,w)
    grad_mu = X*(scipy.stats.norm.pdf(phi,0,1).reshape(-1,1))
    return(np.sum((- Y*(1/(mu)) + (1-Y)*(1/(1+epsilon-mu)))*grad_mu,0) + w).squeeze()

def second_deivative(w,X,y):
    mu = compute_mu(X, w)
    R = np.eye(n)

    phi=np.dot(X,w)
    for i in range(n):
        t1 = (y[i] - mu[i,0])/(mu[i,0] * (1-mu[i,0]))
        t2 = scipy.stats.norm.pdf(phi[i,0],0,1)
        t3 = (1-y[i])/np.power(1-mu[i,0],2) + y[i]/np.power(mu[i,0],2)
        R[i,i] = t1*t2*np.dot(X[i],w) + t3*t2*t2

    return(np.dot(np.dot(np.transpose(X),R),X) + np.eye(d))

def neg_log_posterior(w):
    w=w.reshape(-1,1)
    epsilon = 1e-12
    mu = compute_mu(X, w)
    prob_1 = Y*np.log(mu+epsilon)
    prob_0 = (1-Y)*np.log(1-mu+epsilon)
    log_like = np.sum(prob_1) + np.sum(prob_0)
    w_norm = np.power(np.linalg.norm(w),2)
    neg_log_pos = -log_like+w_norm/2
    print("neg_log_posterior = {:.4f} \tlog_like = {:.4f} \tw_norm = {:.4f}".format(neg_log_pos, log_like, w_norm))
    return(neg_log_pos)

def test(w, X, y):
    n,d = X.shape
    mu = compute_mu(X, w)
    #print(mu.shape, n, d)
    yhat = np.zeros((n,1)).astype(np.float64)
    yhat[mu>0.5]=1
    correct = np.sum(yhat==y)
    return(correct,n)

res = minimize(neg_log_posterior, initialise_w('random'), method='BFGS', jac=first_derivative,
               tol= 1e-5, options={'maxiter': 100})
correct,n = test(res.x, Xtest, Ytest)
print("\n_____________Model trained______________\n")
print("\nModel weights : ", res.x)
print("\n_____________Test Accuracy______________\n")

print("Accuracy : {}% ".format(correct/n*100))

w is initialised from N[0,1]
neg_log_posterior = 8737.0582 	log_like = -8734.3740 	w_norm = 5.3684
neg_log_posterior = 5647.9542 	log_like = -5646.5795 	w_norm = 2.7493
neg_log_posterior = 1804.1227 	log_like = -1803.4229 	w_norm = 1.3996
neg_log_posterior = 3582.6792 	log_like = -3578.6622 	w_norm = 8.0339
neg_log_posterior = 1804.1227 	log_like = -1803.4229 	w_norm = 1.3996
neg_log_posterior = 3582.6792 	log_like = -3578.6622 	w_norm = 8.0339
neg_log_posterior = 758.3798 	log_like = -757.7185 	w_norm = 1.3227
neg_log_posterior = 397.1981 	log_like = -396.7553 	w_norm = 0.8856
neg_log_posterior = 6140.0907 	log_like = -6117.5728 	w_norm = 45.0357
neg_log_posterior = 1268.5175 	log_like = -1267.4807 	w_norm = 2.0736
neg_log_posterior = 299.4804 	log_like = -299.2591 	w_norm = 0.4426
neg_log_posterior = 172.2659 	log_like = -172.0000 	w_norm = 0.5317
neg_log_posterior = 132.7590 	log_like = -132.4435 	w_norm = 0.6311
neg_log_posterior = 116.2467 	log_like = -115.8640 	w_norm = 0.7654
ne

/tmp/ipykernel_13078/1007993727.py:13: RuntimeWarning: overflow encountered in true_divide
  return(np.sum((- Y*(1/(mu)) + (1-Y)*(1/(1+epsilon-mu)))*grad_mu,0) + w).squeeze()


# Part 3

In [100]:
def compute_mu(X, w):
    phi=np.dot(X,w)
    mu = norm.cdf(phi)
    mu = mu.reshape(X.shape[0],1)
    return mu

def first_derivative(w):
    mu = compute_mu(X, w)
    epsilon = 1e-12
    phi=np.dot(X,w)
    grad_mu = X*(scipy.stats.norm.pdf(phi,0,1).reshape(-1,1))
    return(np.sum((- Y*(1/(mu)) + (1-Y)*(1/(1+epsilon-mu)))*grad_mu,0) + w).squeeze()

def second_deivative(w,X,y):
    mu = compute_mu(X, w)
    R = np.eye(n)

    phi=np.dot(X,w)
    for i in range(n):
        t1 = (y[i] - mu[i,0])/(mu[i,0] * (1-mu[i,0]))
        t2 = scipy.stats.norm.pdf(phi[i,0],0,1)
        t3 = (1-y[i])/np.power(1-mu[i,0],2) + y[i]/np.power(mu[i,0],2)
        R[i,i] = t1*t2*np.dot(X[i],w) + t3*t2*t2

    return(np.dot(np.dot(np.transpose(X),R),X) + np.eye(d))

def neg_log_posterior(w):
    w=w.reshape(-1,1)
    epsilon = 1e-12
    mu = compute_mu(X, w)
    prob_1 = Y*np.log(mu+epsilon)
    prob_0 = (1-Y)*np.log(1-mu+epsilon)
    log_like = np.sum(prob_1) + np.sum(prob_0)
    w_norm = np.power(np.linalg.norm(w),2)
    neg_log_pos = -log_like+w_norm/2
    print("neg_log_posterior = {:.4f} \tlog_like = {:.4f} \tw_norm = {:.4f}".format(neg_log_pos, log_like, w_norm))
    return(neg_log_pos)

def test(w, X, y):
    n,d = X.shape
    mu = compute_mu(X, w)
    #print(mu.shape, n, d)
    yhat = np.zeros((n,1)).astype(np.float64)
    yhat[mu>0.5]=1
    correct = np.sum(yhat==y)
    return(correct,n)

def train(initialise):

    np.random.seed(0)
    w = initialise_w(initialise)
    for j in range(100):

        grad1 = first_derivative(w.squeeze()).reshape(d,1)
        H = second_deivative(w, X, Y)
        delta_w = np.dot(np.linalg.inv(H),grad1)
        w = w - delta_w
        diff = np.linalg.norm(delta_w)

        correct,n = test(w, Xtest, Ytest)
        print("Iteration : {} \t Accuracy : {}%".format(j,correct/n*100))

        if(diff < 1e-5):
            print("tolerance reached at the iteration : ",j)
            break
    print("Training done...")
    print("Model weights : ", np.transpose(w))

In [102]:
n,d = X.shape
n1,d1 = Xtest.shape

Y = Y.reshape(n,1)
Ytest = Ytest.reshape(n1,1)

train('zeros')

w is initialised as a zero vector
Iteration : 0 	 Accuracy : 93.8%
Iteration : 1 	 Accuracy : 94.8%
Iteration : 2 	 Accuracy : 94.8%
Iteration : 3 	 Accuracy : 95.0%
Iteration : 4 	 Accuracy : 95.39999999999999%
Iteration : 5 	 Accuracy : 95.6%
Iteration : 6 	 Accuracy : 95.6%
Iteration : 7 	 Accuracy : 95.6%
Iteration : 8 	 Accuracy : 95.6%
Iteration : 9 	 Accuracy : 95.6%
tolerance reached at the iteration :  9
Training done...
Model weights :  [[-1.50015252 -0.90812559 -0.98061604 -0.44317884]]
